In [1]:
import gym
import random
import numpy as np
import time
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from collections import deque
from gym.envs.registration import registry, register
from IPython.display import clear_output

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
try:
    register(
        id='FrozenLakeNoSlip-v0',
        entry_point='gym.envs.toy_text:FrozenLakeEnv',
        kwargs={'map_name' : '4x4', 'is_slippery' : False},
        max_episode_steps=100,
        reward_threshold=0.70, # optimum = 0.74
    )
except:
    pass

env_name = 'FrozenLakeNoSlip-v0'
env = gym.make(env_name)
print(env.observation_space)
print(env.action_space)

Discrete(16)
Discrete(4)


In [3]:
class Agent():
    def __init__(self, env):
        self.is_discrete = type(env.action_space == gym.spaces.discrete.Discrete)
        
        if self.is_discrete:
            self.action_size = env.action_space.n
            print("action size", self.action_size)
        else:
            self.action_low = env.action_space.low
            self.action_high = env.action_space.high
            self.action_shape = env.action_space.shape
            print("action range", self.action_low, self.action_high)
    
    def get_action(self, state):
        if self.is_discrete:
            action = random.choice(range(self.action_size))
        else:
            action = np.random.uniform(self.action_low, 
                                       self.action_high, 
                                       self.action_shape)
#         pole_angle = state[2]
#         action = 0 if pole_angle<0 else 1
        return action

In [4]:
class ExpReplay_QNAgent(Agent):
    def __init__(self, env, discount_rate=0.97, learning_rate=0.001):
        super().__init__(env)
        self.state_size = env.observation_space.n
        print("state size", self.state_size)
        
        self.eps = 1.0
        self.discount_rate = discount_rate
        self.learning_rate = learning_rate
        self.build_model()
        
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
#         push items out of the buffer if we try to overflow it
        self.replay_buffer = deque(maxlen=1000)
        
    def build_model(self):
        tf.reset_default_graph()
        self.state_in = tf.placeholder(tf.int32, shape=[None])
        self.action_in = tf.placeholder(tf.int32, shape=[None])
        self.target_in = tf.placeholder(tf.float32, shape=[None])
        
        self.state = tf.one_hot(self.state_in, depth=self.state_size)
        self.action = tf.one_hot(self.action_in, depth=self.action_size)
        
        self.q_state = tf.layers.dense(self.state, units=self.action_size, name='q_table')
        self.q_action = tf.reduce_sum(tf.multiply(self.q_state, self.action), axis=1)
        
        self.loss = tf.reduce_mean(tf.square(self.target_in - self.q_action))
        self.optimizer = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss)
        
    def get_action(self, state):
        q_state = self.sess.run(self.q_state, feed_dict={self.state_in: [state]})
        action_greedy = np.argmax(q_state)
        action_random = super().get_action(state)        
        return action_random if random.random() < self.eps else action_greedy
    
    def train(self, experience, batch_size=50):
        self.replay_buffer.append(experience)
        samples = random.choices(self.replay_buffer, k=batch_size)
        state, action, next_state, reward, done = (list(col) for col in zip(experience, *samples))
#         state, action, next_state, reward, done = [[exp] for exp in experience]
        
        q_next = self.sess.run(self.q_state, feed_dict={self.state_in: next_state})
        q_next[done] = np.zeros([self.action_size])
        q_target = reward + self.discount_rate * np.max(q_next, axis=1)
        
        feed = {self.state_in: state, self.action_in: action, self.target_in: q_target}
        self.sess.run(self.optimizer, feed_dict=feed)
        
        if experience[4]:
            self.eps = self.eps * 0.99
            
    def __del__(self):
        self.sess.close()
        
agent = ExpReplay_QNAgent(env)        

action size 4
state size 16


/home/rhushabh/.local/lib/python3.8/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/home/rhushabh/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1692: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


In [6]:
total_reward = 0
for ep in range(100):
    state = env.reset()
    done = False
    while not done:
        action = agent.get_action(state)
        next_state, reward, done, info = env.step(action)
        agent.train((state, action, next_state, reward, done))
        state = next_state
        total_reward += reward
        
        print("s", state, "a", action)
        print("Episode: {}, Total Reward: {}, eps: {}".format(ep, total_reward, agent.eps))
        env.render()
        with tf.variable_scope('q_table', reuse=True):
            weights = agent.sess.run(tf.get_variable("kernel"))
            print(weights)
        time.sleep(0.05)
        clear_output(wait=True)
    
env.close()

s 15 a 2
Episode: 99, Total Reward: 76.0, eps: 0.13397967485796175
  (Right)
SFFF
FHFH
FFFH
HFFG
[[ 0.3006737   0.52607274  0.29399785  0.10044216]
 [ 0.29179573 -0.32824036  0.3238878   0.1090266 ]
 [ 0.32995394  0.5843585   0.29563883  0.16134971]
 [ 0.35717335 -0.08917359  0.30126524  0.13873646]
 [ 0.25850734  0.5515119  -0.56129235  0.09852203]
 [-0.17450333 -0.41611284 -0.23121658  0.25742334]
 [-0.27072662  0.61257446 -0.45788103  0.1535136 ]
 [-0.5234525  -0.32778597  0.23234004 -0.46798182]
 [ 0.3118631  -0.2763494   0.34110278  0.13098115]
 [ 0.29688507  0.5634713   0.37712723 -0.5558081 ]
 [ 0.38171446  0.6409006  -0.5230283   0.03534756]
 [ 0.53956866 -0.45406333  0.20857888 -0.45091903]
 [-0.01162958 -0.01381731  0.54129946  0.2815627 ]
 [-0.20188054  0.10262319  0.04433797  0.17696546]
 [ 0.12444116  0.59353393  0.43757805  0.16647089]
 [ 0.34633458 -0.16765586  0.4803394   0.5267333 ]]
